# PyTorch - homework 1


Please run the whole notebook with your code and submit the `.ipynb` file that includes your answers. 

In [1]:
from termcolor import colored

student_number="1004455"
student_name="Victoria Yong"

print(colored("Homework by "  + student_name + ', number: ' + student_number,'red'))

Homework by Victoria Yong, number: 1004455


In [2]:
import os
import torch
import numpy as np
import pandas as pd 

import torch.nn as nn
import torch.nn.functional as F

 ## Question 1 -- matrix multiplication

Implement the following mathematical operation on both the CPU and GPU (use Google Colab or another cloud service if you don't have a GPU in your computer). Print:

a) which type of GPU card you have and 

b) show the computation time for both CPU and GPU (using PyTorch). 

c) How much % faster is the GPU? 

 The operation to implement is the dot product $C = B * A^T$

 whereby $A$ is a random matrix of size $20,000 \times 1,000$ and $B$ is a random matrix of size $2,000 \times 1,000$. In addition to the required information asked above:
 
 d) please also print the resulting two $C$ matrices (they should be the same btw). 
 



In [3]:
# implement solution here
print(torch.cuda.get_device_name(0))


NVIDIA GeForce RTX 3060 Ti


In [4]:
cpu_start = torch.cuda.Event(enable_timing=True)
cpu_end = torch.cuda.Event(enable_timing=True)

A = torch.rand((20000, 1000)).to('cpu')
b = torch.rand((2000, 1000)).to('cpu')

## CPU
cpu_start.record()
c = b@A.t()
cpu_end.record()

# Waits for everything to finish running
torch.cuda.synchronize()

cpu_time = cpu_start.elapsed_time(cpu_end)

print('CPU: ', cpu_time)
print(c)
torch.cuda.empty_cache()

## GPU
gpu_start = torch.cuda.Event(enable_timing=True)
gpu_end = torch.cuda.Event(enable_timing=True)

A.cuda()
b.cuda()

gpu_start.record()
c = b@A.t()
gpu_end.record()

# Waits for everything to finish running
torch.cuda.synchronize()

gpu_time = gpu_start.elapsed_time(gpu_end)
print('GPU: ', gpu_time)
print(c)

print(f"GPU is faster by {(gpu_time - cpu_time)/gpu_time*100}%")

CPU:  161.50930786132812
tensor([[263.2691, 257.2154, 253.2079,  ..., 252.6945, 259.6516, 251.9566],
        [253.8056, 254.6681, 253.5869,  ..., 236.7095, 255.4012, 246.2735],
        [252.6508, 253.7045, 252.9636,  ..., 244.9855, 251.5641, 247.7907],
        ...,
        [255.4564, 253.3973, 255.3647,  ..., 244.7879, 254.4832, 246.4949],
        [260.1990, 257.9343, 255.4839,  ..., 247.3258, 258.9628, 249.9205],
        [247.3627, 248.4945, 251.4022,  ..., 231.2974, 250.6243, 243.7911]])
GPU:  169.88336181640625
tensor([[263.2691, 257.2154, 253.2079,  ..., 252.6945, 259.6516, 251.9566],
        [253.8056, 254.6681, 253.5869,  ..., 236.7095, 255.4012, 246.2735],
        [252.6508, 253.7045, 252.9636,  ..., 244.9855, 251.5641, 247.7907],
        ...,
        [255.4564, 253.3973, 255.3647,  ..., 244.7879, 254.4832, 246.4949],
        [260.1990, 257.9343, 255.4839,  ..., 247.3258, 258.9628, 249.9205],
        [247.3627, 248.4945, 251.4022,  ..., 231.2974, 250.6243, 243.7911]])
GPU is fas

## Question 2 - grad


Find the gradient (partial derivatives) of the function $g(w)$ below. 

Let  $w=[w_1,w_2]^T$

Consider  $g(w)=2w_1w_2+w_2cos(w_1)$

a) In PyTorch, compute:   $\nabla g(w)$ 

 and verify that $\nabla g([\pi,1])=[2,2\pi−1]^T$ using the grad function, whereby the first position is the partial for $w_1$ and the second position is the partial for $w_2$. 

b) You can also write a function to manually calculate these partial derivatives! You can review your differential equations math at [here](https://www.wolframalpha.com/input/?i=derivative+y+cos%28x%29) and implement this as a second function below to verify that it comes to the same solution. 


In [5]:
# write your solution here
w = torch.tensor([np.pi,1.],requires_grad=True).reshape(-1,1)
g_w = 2 * w[0] * w[1]  + w[1] * torch.cos(w[0])

dw_auto = torch.autograd.grad(g_w, w)
print(f'Autograd Partial Differentiation : {dw_auto}')

dw1 = 2 * w[1] - torch.sin(w[0])*w[1]
dw2 = 2 * w[0] + torch.cos(w[0])

dw_manual = torch.Tensor([dw1,dw2])
print(f'Manual partial differntiation : {dw_manual}')

Autograd Partial Differentiation : (tensor([[2.0000],
        [5.2832]]),)
Manual partial differntiation : tensor([2.0000, 5.2832])


## Question 3 - dance hit song prediction

Implement logistic regression in PyTorch for the following dance hit song prediction training dataset: 
https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030training.csv

 * Input variables: a number of audio features (most already standardized so don't worry about that)
 * Target variable: Topclass1030: 
   * 1 means it was a top 10 hit song; 
   * 0 means it never went above top 30 position.

This dataset is derived from my paper on dance hit song prediction, for full description of features have a look at https://arxiv.org/abs/1905.08076. 

Print the evolution of the loss every few epochs and train the model until it converges. 
 
 After training the logistic regression model, calculate the prediction accuracy on the test set: 
 https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030test.csv








In [6]:
# load train_data
train_url = 'https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030training.csv'
test_url = 'https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030test.csv'

train_data = pd.read_csv(train_url)
train_labels = train_data['Topclass1030']
train_feats = train_data.drop('Topclass1030', 1)

# define logistic regression model

class LogisticRegression(nn.Module):
 
  def __init__(self, input_size, num_classes):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(input_size, num_classes)
   

  def forward(self, x):
    out = self.linear(x)
    out = torch.sigmoid(out)
    return out

# train model
num_outputs = 1
num_input_features = train_data.shape[1] - 1
model = LogisticRegression(num_input_features, num_outputs).cuda()

lr_rate = 0.001  # alpha
loss_function = nn.BCELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

epochs = 200 

for i in range(epochs):
    for j in range(train_data.shape[0]):
        feats = torch.tensor(train_feats.loc[j].values).float().cuda()
        label = torch.tensor([train_labels.loc[j]]).float().cuda()

        optimizer.zero_grad()
        pred = model(feats)

        loss = loss_function(pred, label).cuda() 
        loss.backward() 
        optimizer.step() 

    if i % 10 == 0:
        print (f"Epoch: {i}\t Loss: {loss.data.cpu().numpy()}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_16472\724039460.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train_feats = train_data.drop('Topclass1030', 1)


Epoch: 0	 Loss: 0.8439046144485474
Epoch: 10	 Loss: 0.5644068121910095
Epoch: 20	 Loss: 0.4772292375564575
Epoch: 30	 Loss: 0.42795199155807495
Epoch: 40	 Loss: 0.39873531460762024
Epoch: 50	 Loss: 0.3798719346523285
Epoch: 60	 Loss: 0.36710214614868164
Epoch: 70	 Loss: 0.35813388228416443
Epoch: 80	 Loss: 0.35175463557243347
Epoch: 90	 Loss: 0.3472670018672943
Epoch: 100	 Loss: 0.3439001739025116
Epoch: 110	 Loss: 0.34153133630752563
Epoch: 120	 Loss: 0.33975327014923096
Epoch: 130	 Loss: 0.3387387692928314
Epoch: 140	 Loss: 0.3380681574344635
Epoch: 150	 Loss: 0.3379520773887634
Epoch: 160	 Loss: 0.3377499282360077
Epoch: 170	 Loss: 0.3376959562301636
Epoch: 180	 Loss: 0.3381713628768921
Epoch: 190	 Loss: 0.3390369713306427


Run the below code to test the accuracy of your model on the training set: 

In [9]:
import pandas as pd 

test = pd.read_csv(test_url)
labels = test.iloc[:,-1]
test = test.drop('Topclass1030', axis=1)
testdata = torch.Tensor(test.values)
testlabels = torch.Tensor(labels.values).view(-1,1)

TP = 0
TN = 0
FN = 0
FP = 0

for i in range(0, testdata.size()[0]): 
  # print(testdata[i].size())
  Xtest = torch.Tensor(testdata[i]).cuda()
  y_hat = model(Xtest)
  
  if y_hat > 0.5:
    prediction = 1
  else: 
    prediction = 0

  if (prediction == testlabels[i]):
    if (prediction == 1):
      TP += 1
    else: 
      TN += 1

  else:
    if (prediction == 1):
      FP += 1
    else: 
      FN += 1

print("True Positives: {0}, True Negatives: {1}".format(TP, TN))
print("False Positives: {0}, False Negatives: {1}".format(FP, FN))
rate = TP/(FN+TP)
print("Class specific accuracy of correctly predicting a hit song is {0}".format(rate))

True Positives: 39, True Negatives: 20
False Positives: 9, False Negatives: 11
Class specific accuracy of correctly predicting a hit song is 0.78
